In [67]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jul 23 2015

@author: Anton at 0x0af@ukr.net
"""

import numpy
import scipy
import copy, math
from scipy.fftpack import dct, idct
from scipy.linalg import schur
from PIL import Image

In [68]:
def SchurDecomposition(GrayscaleImage2DArray):
    """
     Schur decomposition. Input a grayscale image to get it's upper triangular matrix and unitary matrix.
     
     Schur's theorem announced that : if A ∈ Cn×n, there exists a unitary matrix U and an upper triangular matrix T,
     such that: A = U × T × U'
     
     0x0af@ukr.net, 23-July-2015
    """
    
    TriangularMatrix, UnitaryMatrix = scipy.linalg.schur(GrayscaleImage2DArray)
    
    return TriangularMatrix, UnitaryMatrix

In [74]:
def InverseSchurDecomposition(TriangularMatrix, UnitaryMatrix):
    """
     Inverse Schur decomposition. Input a 2D unitary matrix and a 2D triangular matrix to get the original grayscale image
    
     23-July-2015
    """
    
    OriginalGrayscaleImage2DArray = numpy.dot(UnitaryMatrix , numpy.dot(TriangularMatrix, UnitaryMatrix.conj().T))
    
    return OriginalGrayscaleImage2DArray

In [75]:
def RazafindradinaEmbed(GrayscaleContainerPath,GrayscaleWatermarkPath,WatermarkedImagePath,Alpha):
    """    
    Razafindradina embedding method implementation. 
    
    Outputs the resulting watermarked image
    
    23-July-2015
    """
    
    grayscaleContainer2DArray = numpy.asarray(Image.open(GrayscaleContainerPath).convert("L"))
    grayscaleWatermark2DArray = numpy.asarray(Image.open(GrayscaleWatermarkPath).convert("L"))
    
    assert (grayscaleContainer2DArray.shape[0]==grayscaleContainer2DArray.shape[1]) and (grayscaleContainer2DArray.shape[0]==grayscaleWatermark2DArray.shape[0]) and (grayscaleContainer2DArray.shape[1]==grayscaleWatermark2DArray.shape[1]), 'GrayscaleContainer and GrayscaleWatermark sizes do not match or not square' 
    
    # Perform DCT on GrayscaleContainer
    
    #print grayscaleContainer2DArray
    
    GCDCT = dct(dct(grayscaleContainer2DArray,axis=0,norm='ortho'),axis=1,norm='ortho')
    
    #print grayscaleContainer2DArray
    
    # Perform SchurDecomposition on GrayscaleWatermark
    
    GWSDT, GWSDU = SchurDecomposition(grayscaleWatermark2DArray)
    
    # Alpha-blend GrayscaleWatermark TriangularMatrix into GrayscaleContainer DCT coeffs with Alpha
    
    GCDCT += GWSDT * Alpha
    
    # Perform IDCT on GrayscaleContainer DCT coeffs to get WatermarkedImage
    
    watermarkedImage2DArray = idct(idct(GCDCT,axis=0,norm='ortho'),axis=1,norm='ortho')
    
    watermarkedImage2DArray[watermarkedImage2DArray>255] = 255
    watermarkedImage2DArray[watermarkedImage2DArray<0] = 0
    
    watermarkedImage = Image.fromarray(numpy.uint8(watermarkedImage2DArray))
    watermarkedImage.show()
    
    # Write image to file
    
    watermarkedImage.save(WatermarkedImagePath)
    
    return

In [76]:
def RazafindradinaExtract(GrayscaleStegoPath, GrayscaleContainerPath, GrayscaleWatermarkPath, ExtractedGrayscaleWatermarkPath, Alpha):
    """
    Rezefindradina extracting method implementation.
    
    Outputs the extracted watermark
    
    23-July-2015
    """
    
    grayscaleContainer2DArray = numpy.asarray(Image.open(GrayscaleContainerPath).convert("L"))
    grayscaleStego2DArray = numpy.asarray(Image.open(GrayscaleStegoPath).convert("L"))
    grayscaleWatermark2DArray = numpy.asarray(Image.open(GrayscaleWatermarkPath).convert("L"))
    
    # Perform DCT on GrayscaleContainer
    
    GCDCT = dct(dct(grayscaleContainer2DArray,axis=0,norm='ortho'),axis=1,norm='ortho')
    
    # Perform DCT on GrayscaleStego
    
    GSDCT = dct(dct(grayscaleStego2DArray,axis=0,norm='ortho'),axis=1,norm='ortho')
    
    # Extract ExtractedWatermarkTriangularMatrix with Alpha
    
    EWSDT = (GSDCT - GCDCT) / Alpha
    
    # Perform InverseSchurDecomposition on UnitaryMatrix (non-embedded) and TrinagularMatrix to get ExtractedGrayscaleWatermark
    
    GWSDT, GWSDU = SchurDecomposition(grayscaleWatermark2DArray)
    
    ExtractedWatermark2DArray = InverseSchurDecomposition(EWSDT,GWSDU)
    
    ExtractedWatermark2DArray[ExtractedWatermark2DArray>255] = 255
    ExtractedWatermark2DArray[ExtractedWatermark2DArray<0] = 0
    
    extractedGrayscaleWatermark = Image.fromarray(numpy.uint8(ExtractedWatermark2DArray))
    extractedGrayscaleWatermark.show()
    
    # Write image to file
    
    extractedGrayscaleWatermark.save(ExtractedGrayscaleWatermarkPath)
    
    return

In [82]:
RazafindradinaEmbed("C:\Users\Tony\Documents\GitHub\steganography-embedding\GrayscaleContainer.bmp","C:\Users\Tony\Documents\GitHub\steganography-embedding\SameSizeGrayscaleWatermark.bmp","C:\Users\Tony\Documents\GitHub\steganography-embedding\Razafindradina_Watermarked_Image.bmp",0.01)

In [83]:
RazafindradinaExtract("C:\Users\Tony\Documents\GitHub\steganography-embedding\Razafindradina_Watermarked_Image.bmp","C:\Users\Tony\Documents\GitHub\steganography-embedding\GrayscaleContainer.bmp","C:\Users\Tony\Documents\GitHub\steganography-embedding\SameSizeGrayscaleWatermark.bmp","C:\Users\Tony\Documents\GitHub\steganography-embedding\Razafindradina_Extracted_Watermark.bmp",0.01)